In [3]:
%pip install wget
#%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
import data_rnn
import numpy as np
import torch


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Mark\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Collecting wget
  Using cached wget-3.2-py3-none-any.whl



In [4]:
# Check and set up GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = data_rnn.load_imdb(final=False)

# Part 1: Classification: data loading

In [6]:
x_train[0], y_train[0] # 0 is positive, 1 is negative

([14, 19, 9, 379, 22, 11, 50, 52, 53, 290], 1)

In [7]:
[i2w[word] for word in x_train[0]]
len(i2w)

99430

In [8]:
lengths = [len(l) for l in x_train]
print(np.min(lengths), np.max(lengths), np.average(lengths))

10 2514 240.6318


In [9]:
w2i['.pad'], w2i['.start'], w2i['.end'], w2i['.unk']

(0, 1, 2, 3)

## Question 1: Padding and Conversion

In [10]:
def pad(seq, pad_length):
    padded = np.zeros(pad_length) # 0 is for padding
    padded[0:len(seq)] = seq
    return torch.tensor(padded, dtype=torch.long)

In [11]:
padded = pad(x_train[0], 12).reshape(1,-1)
padded.shape

torch.Size([1, 12])

# Part 2: Classification, baseline model

## Question 2: Baseline model

In [12]:
# Take a tensor x, and return max across time dimension
def MaxPoolTime(x):
    return torch.amax(x, dim=1)

class q2_baseline(torch.nn.Module):
    def __init__(self, batch_size=1):
        super().__init__()
        timestep = 12
        numcls = 2
        hidden = 300
        embedding_size = 300
        n_embeddings = len(i2w)
        self.emb = torch.nn.Embedding(n_embeddings, embedding_size, padding_idx=0)
        self.fc1 = torch.nn.Linear(embedding_size, hidden)
        self.fc2 = torch.nn.Linear(hidden, numcls)
    
    def forward(self, x):
        x = self.emb(x)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = MaxPoolTime(x)
        x = self.fc2(x)

        return x

## Question 3: Training Loop

In [ ]:
# pad all
padding_size = np.max(lengths)

padded_train = torch.stack([pad(x, padding_size) for x in x_train])
padded_val = torch.stack([pad(x, padding_size) for x in x_val])

In [14]:
padded_train.shape

torch.Size([20000, 2514])

Create train and validation datasets

In [29]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 1024

# TO RUN ON THE GPU:
# the 2 lines below are critical.
padded_train = padded_train.to(device)
padded_val = padded_val.to(device)
y_train_tensor = torch.tensor(y_train).to(device)
y_val_tensor = torch.tensor(y_val).to(device)

# create train and val datasets with instance and label pairs
train_dataset = TensorDataset(padded_train, y_train_tensor)
validation_dataset = TensorDataset(padded_val, y_val_tensor)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


Training loop

In [30]:
baseline_model = q2_baseline(batch_size).to(device)

# Setup optimizer
import torch.optim as optim
optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

In [32]:
def validate(model, validation_loader):
    val_acc = 0
    val_correct = 0
    total_samples = 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(validation_loader):
            instances, labels = data
            fwd = model(instances)
            predictions = torch.argmax(fwd, dim=1)
            val_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    # Compute and return accuracy
    val_acc = val_correct / total_samples
    return val_acc

In [ ]:
def train_model(model, trainloader, testloader, optimizer, nr_epochs=5):
    model.train()
    for epoch in range(nr_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, batch in enumerate(trainloader, 0):
            instances, labels = batch

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(instances)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if (i % len(trainloader)/len(batch)*2 == 0):    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
                running_loss = 0.0
        val_acc = validate(model, testloader)
        print(f'Epoch {epoch}, validation acc.: {val_acc}')
    print('Finished Training')
    return model

In [34]:
final_model = train_model(baseline_model, trainloader, testloader, optimizer, 5)

KeyboardInterrupt: 

# Part 3: Writing your own Elman RNN

To be completed!